In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import product

def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

from scipy.stats import binned_statistic_2d

regions=pd.read_csv('regions.csv', sep=';')
    
regions.index=regions.region
regions.drop(['region'], axis=1, inplace=True)

dan=['yellow_tripdata_2016-01.csv', 'yellow_tripdata_2016-02.csv',
     'yellow_tripdata_2016-03.csv', 'yellow_tripdata_2016-04.csv',
     'yellow_tripdata_2016-05.csv', 'yellow_tripdata_2016-06.csv']

def shape0(df):
    return df.shape[0]

cols=[1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132,
       1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182,
       1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230,
       1231, 1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280,
       1281, 1282, 1283, 1284, 1285, 1286, 1287, 1326, 1327, 1331, 1332,
       1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380,
       1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431,
       1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1482, 1483,
       1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069,
       2118, 2119, 2168]

d=pd.DataFrame(columns=cols)

for i in dan:
    data=pd.read_csv(i, parse_dates=[1,2],
                usecols=range(7))

    mask_dist=data.trip_distance>0

    mask_time=(data.tpep_dropoff_datetime>data.tpep_pickup_datetime)

    mask_long=((data.pickup_longitude>=-74.25559) & (data.pickup_longitude<=-73.70001))

    mask_lat=((data.pickup_latitude>=40.49612) & (data.pickup_latitude<=40.91553))

    mask_pas=data.passenger_count>0

    df=data.loc[mask_dist & mask_long & mask_lat & mask_time & mask_pas]

    df.tpep_pickup_datetime=df.tpep_pickup_datetime.dt.floor('1h')

    statistic = binned_statistic_2d(df['pickup_longitude'].values, 
                                   df['pickup_latitude'].values,
                                   df.passenger_count,            #### Тут может использоваться любой столбец
                                   statistic='count', 
                                  bins=[np.union1d(regions.west, regions.east), 
                                        np.union1d(regions.south, regions.north)], 
                                  expand_binnumbers=True)
                                  
    df['region'] = (statistic[3][0] - 1) * 50 + statistic[3][1]

    agg=pd.DataFrame(index=np.unique(df.tpep_pickup_datetime), columns=regions.index.values)

    a=df[['tpep_pickup_datetime', 'region', 'VendorID']].groupby(['tpep_pickup_datetime', 'region']).apply(shape0)

    b=a.unstack()

    b.fillna(0, inplace=True)

    agg.fillna(0, inplace=True)

    c=agg+b

    c.fillna(0, inplace=True)

    c=c.astype(int)
    
    d=d.append(c[cols])

In [11]:
d.to_csv('Taxi.4.csv')

In [8]:
d

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,12,0,2,44,5,41,4,70,7,66
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,29,0,5,2,2,4,0,47,1,29
2016-01-01 02:00:00,90,146,23,110,393,425,313,366,377,138,...,47,0,3,0,4,0,0,69,1,14
2016-01-01 03:00:00,32,87,16,62,252,399,324,309,327,166,...,46,0,2,4,5,1,0,21,0,9
2016-01-01 04:00:00,24,43,10,53,145,254,264,333,318,145,...,43,0,0,1,1,0,0,26,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,116,190,135,132,395,308,401,336,496,260,...,2,44,4,297,311,104,9,142,96,1
2016-06-30 20:00:00,104,142,149,141,333,368,390,385,560,247,...,1,27,7,288,344,103,24,209,145,0
2016-06-30 21:00:00,151,162,145,135,359,422,460,541,672,259,...,2,21,9,287,307,185,9,213,142,1
2016-06-30 22:00:00,106,168,103,125,317,476,405,508,578,259,...,3,19,5,358,387,169,12,206,146,0


In [12]:
pd.read_csv('Taxi.4.csv', index_col=0, parse_dates=True)

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,12,0,2,44,5,41,4,70,7,66
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,29,0,5,2,2,4,0,47,1,29
2016-01-01 02:00:00,90,146,23,110,393,425,313,366,377,138,...,47,0,3,0,4,0,0,69,1,14
2016-01-01 03:00:00,32,87,16,62,252,399,324,309,327,166,...,46,0,2,4,5,1,0,21,0,9
2016-01-01 04:00:00,24,43,10,53,145,254,264,333,318,145,...,43,0,0,1,1,0,0,26,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,116,190,135,132,395,308,401,336,496,260,...,2,44,4,297,311,104,9,142,96,1
2016-06-30 20:00:00,104,142,149,141,333,368,390,385,560,247,...,1,27,7,288,344,103,24,209,145,0
2016-06-30 21:00:00,151,162,145,135,359,422,460,541,672,259,...,2,21,9,287,307,185,9,213,142,1
2016-06-30 22:00:00,106,168,103,125,317,476,405,508,578,259,...,3,19,5,358,387,169,12,206,146,0


### Загрузил данные за половину 2016. Июнь 2016 будет тестовым месяцем

In [2]:
data=data.loc['2016-01-01':'2016-05-31']

In [3]:
data

,1128
2016-01-01 00:00:00,531
2016-01-01 01:00:00,370
2016-01-01 02:00:00,313
2016-01-01 03:00:00,324
2016-01-01 04:00:00,264
...,...
2016-05-31 19:00:00,433
2016-05-31 20:00:00,404
2016-05-31 21:00:00,416
2016-05-31 22:00:00,415
